# 7. 프로젝트 구조 

- 챗봇 프로젝트 디렉터리 구조

- chatbot_test -- train_tools 

               - models --------- intent

               - utils          ㄴ ner

               - config
           
               - test

- train_tools: 챗봇 학습툴 관련 파일 
- models : 챗봇 엔진에서 사용하는 딥러닝 모델 관련 파일
- intent : 의도 분류 모델 관련 파일 
- ner : 개체 인식 모델 관련 파일 
- utils : 챗봇 개발에 필요한 유틸리티 라이브러리 
- config : 챗봇 개발에 필요한 설정 
- test : 챗봇 개발에 필요한 테스트 코드   

## 7-1. 학습용 데이터베이스 설계 및 데이터 테이블 생성 

- 챗봇 엔진 답변 처리 과정에 필요한 DB 구조 설계
- 학습할 수 있는 툴 만들기 

- 학습툴에 필요한 DB 구조 설계 
- 목표: 간단한 수준의 토이 챗봇 - 데이터 무결성, 정규화 큰 신경 X 

| 컬럼   | 속성                      | 설명                                                    |

 : ---     -------------------------    -----------------------------------------------------

| id     | int primary key not null | 학습 데이터 id                                           |

| intent | varchar(45)              | 의도명, 의도가 없는 경우 null                             |

| ner    | varcahr(45)              | 개체명, 개체명이 없는 경우 null                           |

| query  | text null                | 질문 텍스트                                              |

| answer | text not null            | 답변 텍스트                                              |

| answer | image varchar(2048)      | 답변에 들어갈 이미지 URL, 이미지 URL 사용하지 않을 경우 null|

## 7-2. DB 서버 접속 정보를 /config 디럭터리 내 파일로 따로 관리 

- ./config/DatabaseConfig.py 파일 생성

## 7-3. 챗봇 데이터 학습용 테이블 생성 코드 

- 챗복 학습툴과 관련 
- ./train_tools/qna/create_train_data_table.py 생성

In [5]:
# 경로 진행할 때 import sys 하고 진행해야 실행 됨 (다른 디렉터리 에서) 

import pymysql
import sys
from config.DBconfig import *  # DB 접속 정보 불러오기

db = None
try:
    db = pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )

    # 테이블 생성 sql 정의
    sql = '''
      CREATE TABLE IF NOT EXISTS `chatbot_train_data` (
      `id` INT UNSIGNED NOT NULL AUTO_INCREMENT,
      `intent` VARCHAR(45) NULL,
      `ner` VARCHAR(1024) NULL,
      `query` TEXT NULL,
      `answer` TEXT NOT NULL,
      `answer_image` VARCHAR(2048) NULL,
      PRIMARY KEY (`id`))
    ENGINE = InnoDB DEFAULT CHARSET=utf8
    '''

    # 테이블 생성
    with db.cursor() as cur:
        cur.execute(sql)

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()

name 'DB_PASSWORD' is not defined


## 1-4. 챗봇 학습 데이터 엑셀 파일 및 DB 연동 

- 지금 제작하는 학습툴은 화면이 없어서 엑셀 통해 학습 데이터 추가하거나 삭제
- 엑셀 파일 학습툴에 입력해 DB 내용을 업데이트하는 형태
- train_data.xlsx 구조
    - 의도(intent): 질문 의도를 나타내는 텍스트. 의도가 없는 경우 비워둠
    - 개체명 인식(NER): 질문에 필요한 개체명, 개체명이 없는 경우 비워둠
    - 질문(query): 질문 테스트
    - 답변(Answer): 답변 테스트
    - 답변 이미지: 답변에 들어갈 이미지 URL. 이미지 URL이 없는 경우 비워둠


In [2]:
# 엑셀파일 읽어와 DB와 데이터를 동기화하는 코드.
# 챗봇 학습툴 디렉터리
# 챗봇 학습 데이터 불러오기 - load_train_data.py 만들기 

import pymysql
import openpyxl 
import sys 

from config.DBconfig import *    # DB 접속 정보 불러오기 

# 학습 데이터 초기화
def all_clear_train_data(db): 
    # 기존 학습 데이터 삭제
    sql = '''
        delete from chatbot_train_data       
    '''
    with db.cursor() as cur: 
        cur.execute(sql) 

    # auto increament 초기화
    sql = '''
        ALTER TABLE chatbot_train_data AUTO_INCREMENT=1
    '''
    with db.cursor() as cur: 
        cur.execute(sql) 

# db에 데이터 저장 
def insert_data(db, xls_row): 
    intent, ner, query, answer, answer_img_url = xls_row 

    sql = '''
        INSERT chatbot_train_data(intent, ner, query, answer, answer_image)
        values(
            '%s','%s','%s','%s','%s'
        )
    ''' % (intent.value, ner.value, query.value, answer.value, answer_img_url.value) 

    # 엑셀에서 불러온 cell에 데이터가 없는 경우 null로 치환
    sql = sql.replace("'None'", "null") 

    with db.cursor() as cur:
        cur.execute(sql) 
        print('{} 저장'.format(query.value)) 
        db.commit() 

train_file = './train_data.xlsx' 
db = None 
try:
    db = pymysql.connect(
        host = DB_HOST,
        user = DB_USER,
        passwd = DB_PASSWORD,
        db = DB_NAME, 
        charset='utf8'
    )

    # 기존 학습 데이터 초기화 
    # load_train_data.py 프로그램 실행할 때마다 엑셀 파일 내부 데이터와 DB 내 학습 데이터를 동일하게 
    # 유지하기 위해 DB 데이터를 초기화 함.
    # 이 경우 매번 DB 데이터를 지우고 새로 데이터를 입력하는 구조여서 추후 개선 필요 
    # delete 명령어 사용해 챗봇 학습 데이터 테이블 내용 삭제한 후 auto increment 속성 1로 초기화 (AUTO INCREMENT=1) 
    all_clear_train_data(db) 

    # 학습 엑셀 파일 불러오기 
    # openpyxl 모듈 이용해 엑셀 파일 읽어와 DB에 데이터 저장 
    wb = openpyxl.load_workbook(train_file) 
    sheet = wb['Sheet1']
    for row in sheet.iter_rows(min_row=2):    # 헤더는 불러오지 않음 
        # 데이터 저장 
        insert_data(db, row) 

    wb.close() 

except Exception as e: 
    print(e) 

finally:
    if db is not None: 
        db.close()  

'NoneType' object has no attribute 'encoding'


# 8. 챗봇 엔진 만들기

## 8.1 챗봇 엔진 

- 챗봇에서 핵심 기능을 하는 모듈
- 화자 질문 이해하고 알맞은 답변을 출력하는 역할 = 자연어 처리 모듈 
- 카카오톡, 네이버톡톡 봇 빌더 

## 8.2 챗봇 엔진 구조 

- 엔진 설계 전에 만들려는 챗복의 목적과 어떤 도메인 지식을 가지는 챗봇을 만들 것인지 결정 
    - 챗봇 엔진 개발 방법론, 학습에 필요한 데이터셋이 달라짐 
    - 음식 예약 및 주문을 도와주는 음식점 예약 주문에 특화된 챗봇 엔진 실습

- 토이 수준 챗봇 엔진 기능 = 5가지 
    - 질문 의도 분류: 화자 질문 의도 파악. 의도 분류 모델 이요해 의도 클래스 예측 
    - 개체명 인식: 화자 질문에서 단어 토큰별 개체명 인식. 단어 토큰에 맞는 개체명 예측하는 문제
    - 핵심 키워드 추출: 화자 질문 의미에서 핵심이 될 만한 단어 토큰 추출. 형태소 분석기 이용 핵심 키워드 되는 명사나 동사 추출
    - 답변 검색: 해당 질문 의도. 개체명, 핵심 키워드 등을 기반으로 답변을 학습 DB 에서 검색 
    - 소켓 서버: 다양한 종류(카카오톡, 네이버톡톡) 챗봇 클라이언트에서 요청하는 질문 처리 위해 소켓 서버 프로그램 역할. = 챗봇 엔진 서버 프로그램 

- 챗봇 엔진 처리 과정
    1. 질문 문장 입력 - 챗봇엔진 = 전처리 > 형태소 분석기 이용 단어 토큰(키워드) 추출하고 명사나 동사 등 필요 품사만 남기고 불용어 제거 
    2. 의도 분석과 개체명 인식 완료
    3. 결괏값 이용 적적한 답변 학습 DB 에서 검색
    4. 화자에게 답변 출력 

- 자연어 처리 위한 2가지 딥러닝 모델(의도분석, 개체명 인식) 
- 도메인 지식에 맞는 딥러닝 모델 학습 데이터셋을 많이 보유하면 성능 우수한 챗봇 엔진 개발 도움 
- 룰 베이스 + 딥러닝 모델 같이 사용하는 챗봇 엔진 

## 8.3 전처리 과정 

- 형태소 분석기로 토크나이징 작업 하고, 문장 해석에 의미 있는 정보만 남기고 나머지 불용어 들은 제거 

In [ ]:
# 챗봇 엔진 전처리 모듈(클래스) 만들기
# preprocess.py
# 클래스로 정의 - 자주 사용 

from konlpy.tag import Komoran 

class Preprocess:
    def __init__(self, userdic=None):    # 생성자 
        # 형태소 분석기 초기화 
        # Preprocess 클래스 생성될 때 형태소 분석기 인스턴스 객체 생성.
        # 형태소 분석기 = komoran 
        # userdic 인자에 사용자 정의 사전 파일 경로 입력 가능 
        self.komoran = Komoran(userdic=userdic) 

        # 제외할 품사(불용어 정의) 
        # 참조: https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거 
        # 어미 제거 
        # 접미사 제거 
        # 클래스 멤버 변수 exclusion.tags 리스트에 정의 
        self.exclusion_tags = [
            'JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC',
            'SF','SP','SS','SE','SO',
            'EP','EF','EC','ETN','ETM',
            'XSN','XSV','XSA'
        ]

    # 형태소 분석기 POS 태거 
    # Preprocess 클래스 외부에서 코모란 형태소 분석기 객체를 직접 호출할 일이 없게 하기 위해 정의한 래퍼 함수 
    # 형태소 분석기 종류를 바꾸게 될 경우 이 래퍼 함수 내용만 변경하면 됨 = 유지보수 장점 
    def pos(self, sentence):
        return self.komoran.pos(sentence) 

    # 불용어 제거 후 필요한 핵심 키워드(품사) 정보만 가져오기 
    # exclusion_tags 리스트에 해당하지 않는 품사 정보만 키워드로 저장 
    def get_keywords(self, pos, without_tag = False): 
        f = lambda x: x in self.exclusion_tags 
        word_list = [] 
        for p in pos: 
            if f(p[1]) is False: 
                word_list.append(p if without_tag is False else p[0]) 
        return word_list 

In [5]:
# /test.preprocess_test.py 파일 생성
# preprocess 클래스 동작 테스트 

from utils.preprocess import *

sent = '내일 오전 10시에 탕수육 먹고 싶어' 

# 전처리 객체 생성 
p = Preprocess(userdic='../utils/user_dic.tsv')

# 형태소 분석기 실행 
pos = p.pos(sent) 

# 품사 태그와 같이 키워드 출력 
ret = p.get_keywords(pos, without_tag=False) 
print(ret) 

# 품사 태그 없이 키워드 출력
ret = p.get_keywords(pos, without_tag=True) 
print(ret) 

[('내일', 'NNG'), ('오전', 'NNP'), ('10', 'SN'), ('시', 'NNB'), ('탕수육', 'NNP'), ('먹', 'VV'), ('싶', 'VX')]
['내일', '오전', '10', '시', '탕수육', '먹', '싶']


## 8.4 단어 사전 구축 및 시퀀스 생성 

- 의도 분류 및 개체명 인식 모델의 학습을 하려면 단어 사전을 구축해야 함
- 말뭉치 데이터(corpus.txt) /train_tools/dict 디렉터리 
- create_dict.py

In [12]:
# 단어 사전 생성
# 챗봇에서 사용하는 사전 파일 생성 

from utils.preprocess import Preprocess 
from tensorflow.keras import preprocessing 
import pickle 

# 말뭉치 데이터 읽어오기 
def read_corpus_data(filename): 
    with open(filename, 'r', encoding='utf8') as f:
        data = [line.split('\t') for line in f.read().splitlines()] 
        data = data[1:]   # 헤더 제거 
    return data 

# 말뭉치 데이터 가져오기 
# 말뭉치 파일 가져와 리스트로 반환
# corpus.txt = 네이버 영화 리뷰 말뭉치 데이터 기반 데이터 
# 라인마다 tab (\t) 기준 데이터 분리
corpus_data = read_corpus_data('./train_tools/dict/corpus.txt')

# 말뭉치 데이터에서 키워드만 추출해서 사전 리스트 생성 
# 문장 하나씩 불러와 POS 태깅
# 형태소 분석 결과를 단어 리스트(dict) 에 저장 
p = Preprocess() 
dict = [] 
for c in corpus_data: 
    pos = p.pos(c[1]) 
    for k in pos: 
        dict.append(k[0]) 

# 사전에 사용될 word2index 생성 
# 사전의 첫 번째 인덱스에는 OOV 사용 
tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') 
tokenizer.fit_on_texts(dict) 
word_index = tokenizer.word_index 

# 사전 파일 새성
f = open("chatbot_dic.bin", "wb") 
try:
    pickle.dump(word_index, f) 
except Exception as e:
    print(e) 

finally:
    f.close() 


In [18]:
# 단어 사전 테스트 코드 

import pickle 
from utils.preprocess import Preprocess 

# 단어 사전 불러오기 
f = open("./train_tools/dict/chatbot_dic.bin", "rb") 
word_index = pickle.load(f) 
f.close() 

sent = "내일 오전 10시에 짜장면 먹고 싶어 ㅋㅋ" 

# 전처리 객체 생성 
p = Preprocess(userdic='./utils/user_dic.tsv') 

# 형태소 분석기 실행 
pos = p.pos(sent) 

# 품사 태그 없이 키워드 출력 
keywords = p.get_keywords(pos, without_tag=True) 
for word in keywords:
    try:
        print(word, word_index[word]) 

    except KeyError:
        # 해당 단어가 사전에 없는 경우 OOV 처리 
        print(word, word_index['OOV']) 

# 단어 시퀀스 벡터 크기 
MAX_SEQ_LEN = 15 

def GlobalParams(): 
    global MAX_SEQ_LEN

내일 14
오전 269
10시 1
짜장면 527
먹 233
싶 11
ㅋㅋ 10728


In [ ]:
# 입력한 문장 단어 인덱스 사전 이용해 단어 시퀀스 벡터로 변환하는 기능 추가 
# 챗봇 엔진 전처리 과정 포함 
# Preprocess.py 클래스의 메서드 
from konlpy.tag import Komoran
import pickle
import jpype


class Preprocess:
    def __init__(self, word2index_dic='', userdic=None):
        # 단어 인덱스 사전 불러오기
        if(word2index_dic != ''):
            f = open(word2index_dic, "rb")
            self.word_index = pickle.load(f)
            f.close()
        else:
            self.word_index = None

        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC',
            'SF', 'SP', 'SS', 'SE', 'SO',
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            'XSN', 'XSV', 'XSA'
        ]

    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        jpype.attachThreadToJVM()
        return self.komoran.pos(sentence)

    # 불용어 제거 후, 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list

    # 키워드를 단어 인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []

        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                # 해당 단어가 사전에 없는 경우, OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i

## 8.5 의도 분류 모델 

- 챗봇 엔진에 화자 질의 입력 되면 전처리 과정 거치고 해당 문장 의도 분류 
- 문장을 의도 클래스 별로 분류하기 위해 CNN 모델 사용
- 인사, 욕설, 주문, 예약, 기타 

- GlobalParams.py 

In [ ]:
# GlobalParams.py 
# 단어 시퀀스 벡터 크기 
MAX_SEQ_LEN = 15 

def GlobalParams():
    global MAX_SEQ_LEN

### 8.5.1 의도 분류 모델 학습 

- total_train_data.csv  학습 데이터셋 
- 음식점 주문과 예약을 위한 챗봇 특화 
- 의도 분류 모델 생성 및 학습 코드 

In [24]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# 데이터 읽어오기
# CNN모델 학습 위해 query, intent 데이터를 quries 와 intents 리스트에 저장 
# 챗본 전처리 모듈 preprocess1 단어 시퀀스 생성 - 매치오디는 번호로 시퀀스 생성
train_file = "./models/intent/total_train_data.csv"
data = pd.read_csv(train_file, delimiter=',')
queries = data['query'].tolist()
intents = data['intent'].tolist()

from utils.preprocess1 import Preprocess
# p = Preprocess(word2index_dic='../../train_tools/dict/chatbot_dict.bin',
#                userdic='../../utils/user_dic.tsv')
p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dic.bin',
               userdic='./utils/user_dic.tsv')


# 단어 시퀀스 생성
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)


# 단어 인덱스 시퀀스 벡터 생성
# 단어 시퀀스 벡터 크기
# 단어 시퀀스 벡터 크기 동일하게 맞추기 위해 MAX_SEQ_LEN 크기만큰 시퀀스 벡터 패딩 처리 
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# (105658, 15)
print(padded_seqs.shape)
print(len(intents)) #105658

# 학습용, 검증용, 테스트용 데이터셋 생성 
# 학습셋:검증셋:테스트셋 = 7:2:1
# 패딩 처리된 시퀀ㅅ(padded_seqs) 벡터 리스트와 의도(intent) 리스트 전체 데이터셋 객체로 만듦 
# 데이터 랜덤으로 섞고 학습용, 검증용, 테스트용 나누고 실제 학습에 필요한 데이터셋 객체 각각 분리 
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수


# CNN 모델 정의  
# 의도분류 모델은 케라스 함수형 모델 방식으로 구현
# 입력하는 문장을 의도 클래스로 분류하는 CNN모델은 
# 전처리된 입력 데이터를 단어 임베딩 처리하는 영역,
# 합성곱 필터와 연산을 통해 문장의 특징 정보(특징맵) 를 추출하고 평탄화 하는 영역,
# 완전 연결 계층(fully connected layer) 통해 감정별로 클래스 분류하는 영역
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

# 5가지 의도 클래스 분류해야 해서 출력 노드가 5개인 Dense 계층 생성
# 신경망 예측 최종단계라 활성화 함수 사용 안함
# 결과로 나온 값 logits = 점수
# 출력 노드에서 5개 점수 출력 가장 큰 점수 가진 노드 위치가 CNN 모델이 예측한 의도 클래스가 됨
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(5, name='logits')(dropout_hidden)
# 마지막 출력노드로 정의한 logits 에서 나온 점수를 소프트맥스 계츧을 통해 감정 클래스별 확률 계산 
predictions = Dense(5, activation=tf.nn.softmax)(logits)


# 모델 생성 
# 위에서 정의한 계층들을 케라스 모델에 추가하는 작업
# Model 인자(입력계층, 출력계층) 사용. 
# 실제 모델을 model.compile() 함수 통해 CNN 모델 컴파일 
# 최적화 방법 = 'adam', 손실 함수 = sparse_categorical_crossentropy   
# 모델 평가 = 정확도 accoracy  
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# CNN 모델 학습 
# 첫번인자 = 학습용 데이터셋, 두번째 validation_data = 검증용 데이터셋, 에포크=5
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)


# 모델 성능 평가(테스트 데이터 셋 이용)
# 인자: 테스트용 데이터셋 
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))


# 모델 저장  ○8
model.save('intent_model.h5')


(105658, 15)
105658
Epoch 1/5
3698/3698 [==============================] - 69s 18ms/step - loss: 0.0552 - accuracy: 0.9824 - val_loss: 0.0128 - val_accuracy: 0.9960
Epoch 2/5
3698/3698 [==============================] - 69s 19ms/step - loss: 0.0157 - accuracy: 0.9949 - val_loss: 0.0086 - val_accuracy: 0.9959
Epoch 3/5
3698/3698 [==============================] - 66s 18ms/step - loss: 0.0116 - accuracy: 0.9959 - val_loss: 0.0079 - val_accuracy: 0.9971
Epoch 4/5
3698/3698 [==============================] - 70s 19ms/step - loss: 0.0098 - accuracy: 0.9962 - val_loss: 0.0064 - val_accuracy: 0.9972
Epoch 5/5
529/529 [==============================] - 1s 1ms/step - loss: 0.0071 - accuracy: 0.9975
Accuracy: 99.753904
loss: 0.007075


## 8.5.2 의도 분류 모듈 생성 

- 챗봇 엔진 의도 분류 모듈 만들기
- 학습한 의도 분류 모델 파일을 활용해 입력되는 텍스트의 의도 클래스를 예측하는 기능 
- /models/intent.IntentModel.py 생성

In [25]:
# # 챗봇 엔진 의도 분류 모델 모듈
# import tensorflow as tf 
# from tensorflow.keras.models import Model, load_model 
# from tensorflow.keras import preprocessing 

# # 의도 분류 모델 모듈 
# class IntentModel: 
#     def __init__(self, model_name, preprocess): 
#         # 의도 클래스별 레이블 
#         self.labels = {0: "인사", 1: "욕설", 2: "주문", 3: "예약", 4: "기타"}

#         # 의도 분류 모델 불러오기 
#         self.model = load_model(model_name) 

#         # 챗봇 Preprocess 객체 
#         self.p = preprocess 

#     # 의도 클래스 예측 
#     def predict_class(self, query): 
#         # 형태소 분석 
#         pos = self.p.pos(query) 

#         # 문장 내 키워드 추출(불용어 제거) 
#         keywords = self.p.get_keywords(pos, without_tag=True) 
#         sequences = [self.p.get_wordidx_sequence(keywords)] 

#         # 단어 시퀀스 벡터 크기 
#         from config.GlobalParams import MAX_SEQ_LEN 

#         # 패딩 처리 
#         padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post') 
        
#         predict = self.model.predict(padded_seqs) 
#         predict_class = tf.math.argmax(predict, axis = 1) 
#         return predict_class.numpy()[0] 

In [29]:
# IntentModel 클래스 테스트 코드 
# IntentModel 객체 생성해 새로운 유형의 문장 분류 

from utils.preprocess1 import Preprocess 
from models.intent.IntentModel import IntentModel 

# p = Preprocess(word2index_dic='../train_tools/dict/chatbo_dic.bin',
#                userdic = '../utils/user_dic.tsv') 
p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dic.bin',
               userdic = './utils/user_dic.tsv') 

# intent = IntentModel(model_name = '../models/intent/intent_model.h5', preprocess=p)
intent = IntentModel(model_name = './models/intent/intent_model.h5', preprocess=p) 
query = '오늘 육개장 주문 가능한가요?'
predict = intent.predict_class(query) 
predict_label = intent.labels[predict] 

print(query) 
print('의도 예측 클래스 : ', predict) 
print('의도 예측 레이블 : ', predict_label)

AttributeError: 'IntentModel' object has no attribute 'model_predict'

## 8.6 개체명 인식 모델 학습 

- 챗봇 엔진에 입력된 문장의 의도가 분류된 후 문장 내 개체명 인식을 진행 
- 개체명 인식 LSTM 모델 사용 
- 주요 개체명
    - B_FOOD: 음식
    - B_DT.B_TI: 날짜.시간
    - B_PS: 사람 
    - B_OG: 조직, 회사 
    - B_LC: 지역 

### 8.6.1 개체명 인식 모델 학습 

- ner_trian.txt = 학습 데이터셋 
- NER 모델 생성하고 학습하는 코드 
- train_model.py 생성

In [30]:
# 챗봇 엔진 NER 모델 

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from utils.preprocess1 import Preprocess

# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dic.bin',
               userdic='./utils/user_dic.tsv')

# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('./models/ner/ner_train.txt')

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
# 단어(w[1]), BIO 태그(w[3]) 
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    
    sentences.append(sentence)
    tags.append(bio_tag)


print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

# 토크나이저 정의
# 단어 시퀀스의 경우 preprocess1 객체에서 생성하기 때문에 BIO 태그용 토크나이저 객체만 생성
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

# 단어사전 및 태그 사전 크기 정의
# 생성된 사전 리스트 이용해 
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

# 학습용 단어 시퀀스 생성
# 모델에 입력될 문장의 경우 preprocess1에서 생성한 단어 인덱스 시퀀스를 사용
# BIO태그는 위에서 만들어진 사전 데이터를 시퀀스 번호 형태로 인코딩
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환 하기 위해 사용
index_to_ner[0] = 'PAD'

# 시퀀스 패딩 처리
# 개체명 인식 모델의 입출력 벡터 크기를 동일하게 맞추기 위해 시퀀스 패딩 작업 
# 벡터 크기를 위에서 계산한 단어 시퀀스 평균 길이보다 넉넉하게 
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 학습 데이터와 테스트 데이터를 8:2의 비율로 분리
# 8:2 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                                    test_size=0.2,
                                                    random_state=1234)

# 출력 데이터를 one-hot encoding
# 태그 사전 크기에 맞게 
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)


# 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

# 개체 인식 모델 순차 모델 방식 구현
# tag_size 만큼의 출력 뉴런에서 제일 확률 높은 출력값 1개를 선택하는 문제라 softmax 활성화 함수 사용 
# 손실함수 categorical_crossentropy
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)

print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save('ner_model.h5')  # 챗봇 엔진의 개체명 인식 모듈에서 사용


# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
        result.append(temp)
    return result


# f1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report

# 테스트 데이터셋의 NER 예측
# F1 스코어 계산하기 위해 모델의 predict() 함수 통해 테스트용 데이터셋 결과 예측 
# 해당 함수 입력은 시퀀스 번호로 인코딩된 테스트용 단어 시퀀스(넘파이 배열) 사용 
# 해당 함수 결과는 예측된 NER 태그 정보가 담긴 넘파이 밸열이 반환 됨
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test)    # 실제 NER

# F1 평가 결과
# seqeval.metrics 모듈의 classification_report() 함수 통해
# NER 태그별 계산된 정밀도, 재현율, F1스코어 출력
# f1_score() 함수 통해 f1 스코어 값만 출력 가능 
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))


샘플 크기 : 
 61999
0번 째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번 째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 : 168
샘플 단어 시퀀스 평균 길이 : 8.796238649010467
BIO 태그 사전 크기 : 10
단어 사전 크기 : 17751
학습 샘플 시퀀스 형상 :  (49599, 40)
학습 샘플 레이블 형상 :  (49599, 40, 10)
테스트 샘플 시퀀스 형상 :  (12400, 40)
테스트 샘플 레이블 형상 :  (12400, 40, 10)
Epoch 1/10
388/388 [==============================] - 125s 304ms/step - loss: 0.0257 - accuracy: 0.9668
Epoch 2/10
388/388 [==============================] - 140s 362ms/step - loss: 0.0085 - accuracy: 0.9873
Epoch 3/10
388/388 [==============================] - 146s 376ms/step - loss: 0.0056 - accuracy: 0.9914
Epoch 4/10
388/388 [==============================] - 146s 377ms/step - loss: 0.0045 - accuracy: 0.9930
Epoch 5/10
388/388 [==============================] - 148s 381ms/step - loss: 0.0038 - accuracy: 0.9941
Epoch 6/10
388/388 [==============================] - 150s 386ms/step - loss: 0.0033 - accuracy: 0.9947
Epoch 7/10
388/388 [===============

c:\Users\gnjnm\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gnjnm\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gnjnm\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gnjnm\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gnjnm\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gnjnm\anaconda3\lib\site-packages\seqeval\metrics\sequ

              precision    recall  f1-score   support

          NP       1.00      1.00      1.00       303
           _       0.53      0.53      0.53       658
         _DT       0.99      1.00      1.00     13683
       _FOOD       1.00      1.00      1.00     11655
         _LC       0.74      0.57      0.65       314
         _OG       0.65      0.46      0.54       460
         _PS       0.55      0.63      0.59       396
         _TI       0.65      0.72      0.68        61

   micro avg       0.97      0.97      0.97     27530
   macro avg       0.76      0.74      0.75     27530
weighted avg       0.97      0.97      0.97     27530

F1-score: 96.9%


### 8.6.2 개체명 인식 모듈 생성 

- 챗봇 엔진 개체명 인식 모듈 만들기
- 개체명 인식 모델 파일 활용해 입력한 문장 내부의 개체명 인식 기능 가짐 
- /models/ner.NerModel.py 소스 파일 생성

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


# 개체명 인식 모델 모듈
class NerModel:
    def __init__(self, model_name, preprocess):

        # BIO 태그 클래스 별 레이블
        self.index_to_ner = {1: 'O', 2: 'B_DT', 3: 'B_FOOD', 4: 'I', 5: 'B_OG', 6: 'B_PS', 7: 'B_LC', 8: 'NNP', 9: 'B_TI', 0: 'PAD'}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = preprocess


    # 개체명 클래스 예측
    def predict(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding="post", value=0, maxlen=max_len)

        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=-1)

        tags = [self.index_to_ner[i] for i in predict_class.numpy()[0]]
        return list(zip(keywords, tags))

    def predict_tags(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding="post", value=0, maxlen=max_len)

        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=-1)

        tags = []
        for tag_idx in predict_class.numpy()[0]:
            if tag_idx == 1: continue
            tags.append(self.index_to_ner[tag_idx])

        if len(tags) == 0: return None
        return tags

In [2]:
# NerModel 클래스 테스트 코드 
# NerModel 객체 생성해 새로운 유형의 문장에서 개체명 인식 
# /test.model_ner_test.py

from utils.preprocess1 import Preprocess 
from models.ner.NerModel import NerModel 

p = Preprocess(word2index_dic = './train_tools/dict/chatbot_dic.bin', 
               userdic = './utils/user_dic.tsv') 

ner = NerModel(model_name='./models/ner/ner_model.h5', preprocess=p) 
query = '오늘 오전 13시 2분에 돈까스 먹고 싶은데 주문 할 수 있나요?' 
predicts = ner.predict(query) 
print(predicts) 

[('오늘', 'B_DT'), ('오전', 'B_DT'), ('13시', 'B_DT'), ('2분', 'B_DT'), ('돈까스', 'B_FOOD'), ('먹', 'O'), ('싶', 'O'), ('주문', 'O'), ('하', 'O'), ('수', 'O'), ('있', 'O')]


## 8.7 답변 검색 

- 화자로부터 입력된 문장이 전처리, 의도 분류, 개체명 인식 과정을 거쳐 해석된 데이터 기반으로 적절한 답변을 학습 DB로 부터 검색하는 방법
- 챗봇 엔진이 자연어 처리 통해 해석한 문장을 토대로 유사한 답변을 검색하는일은 중요 
- SQL 구문 이용 룰 베이스 기반으로 답변을 검색하는 방법 소개
- 입력 되는 문장 해석 = 딥러닝, 해석 결과 기반 답변 찾는 과정 = 룰 베이스 기반 

### 8.7.1 데이터베이스 제어 모듈 생성 

- 데이터 베이스 제어를 쉽게 할 수 있는 모듈 만들기 
- /utils.Database.py 

In [3]:
# 데이터베이스 제어 모듈

import pymysql
import pymysql.cursors
import logging


class Database:
    '''
    database 제어
    '''

    def __init__(self, host, user, password, db_name, charset='utf8'):
        self.host = host
        self.user = user
        self.password = password
        self.charset = charset
        self.db_name = db_name
        self.conn = None

    # DB 연결
    def connect(self):
        if self.conn != None:
            return

        self.conn = pymysql.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            db=self.db_name,
            charset=self.charset
        )

    # DB 연결 닫기
    def close(self):
        if self.conn is None:
            return

        if not self.conn.open:
            self.conn = None
            return
        self.conn.close()
        self.conn = None

    # SQL 구문 실행
    def execute(self, sql):
        last_row_id = -1
        try:
            with self.conn.cursor() as cur:
                cur.execute(sql)
            self.conn.commit()
            last_row_id = cur.lastrowid
            # logging.debug("excute last_row_id : %d", last_row_id)
        except Exception as ex:
            logging.error(ex)

        finally:
            return last_row_id

    # SELECT 구문 실행 후, 단 1개의 데이터 ROW만 불러옴
    def select_one(self, sql):
        result = None

        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cur:
                cur.execute(sql)
                result = cur.fetchone()
        except Exception as ex:
            logging.error(ex)

        finally:
            return result

    # SELECT 구문 실행 후, 전체 데이터 ROW만 불러옴
    def select_all(self, sql):
        result = None

        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cur:
                cur.execute(sql)
                result = cur.fetchall()
        except Exception as ex:
            logging.error(ex)

        finally:
            return result

### 8.7.2 답변 검색 모듈 생성 

- 입력되는 문장을 전처리, 의도 분류, 개체명 인식 과정 거쳐 나온 자연어 해석 결과를 이용해 학습 DB 에서 적적한 답변을 검색 
- 해석된 결과 항목에 따라 학습 DB 에서 어떤 방식으로 답변을 검색할지 결정하는 일은 챗봇 엔진 설계자 몫 
- 의도명과 개체명 2가지 항목으로만 답변 검색 
- /utils.FindAnswer.py

In [15]:
# 챗봇 답변 검색 모듈 

class FinAnswer:
    # FindAnswer 클래스 생성자.  - Database 인스턴스 객체 인자로 받아 클래스 멤버 변수로 저장. 
    # 이 객체로 답변 검색 
    def __init__(self, db):
        self.db = db 

    # 검색 쿼리 생성
    # 의도명만 검색할지, 여러 종류 개체명 태그와 함께 검색할지 결정하는 조건 
    def _make_query(self, intent_name, ner_tags):
        sql = "select * from 'chatbot_train_data'"
        if intent_name != None and ner_tags == None:
            sql = sql + " where intent={} ".format(intent_name)

        elif intent_name != None and ner_tags != None:
            where = " where intent=%s " % intent_name
            if len(ner_tags) > 0:
                where += 'and (' 
                for ne in ner_tags: 
                    where += " ner like '%{}%' or ".format(ne)
                where = where[:-3] + ')'
            sql = sql + where

        # 동일한 답변이 2개 이상인 경우, 랜덤으로 선택
        sql = sql + ' order by rand() limit 1'
        return sql 

    # 답변 검색 
    # 의도명(intent_name) 개체명 태그 리스트(ner_tags) 이용해 질문 답변 검색하는 메서드 
    # 인자로 제공된 2가지(의도명, 개체명 태그 리스트) 검색시 실패할 수 있음
    # 이런 경우 의도명만 이용해 답변 검색 
    # 정확한 조건 답변 없으면 차선책으로 동일 의도 가지는 답변만 검색 
    def search(self, intent_name, ner_tags): 
        # 의도명과 개체명으로 답변 검색 
        sql = self._make_query(intent_name, ner_tags) 
        answer = self.db.select_one(sql) 

        # 검색되는 답변 없으면 의도명만 검색 
        if answer is None: 
            sql = self._make_query(intent_name, None) 
            answer = self.db.select_one(sql) 

        return (answer['answer'], answer['answer_image']) 

    # NER 태그를 실제 입력한 단어로 변환 
    # 예: '자장면 주문할게요' 텍스트 챗봇 엔진 입력 
    # 챗봇엔진은 자장면 = B_FOOD 개체명 인식 
    # 검색된 답변이 '{B_FOOD} 주문 처리 완료 되었습니다. 주문해 주셔서 감사합니다' 
    # 답변 내용 속 {B_FOOD} 자장면으로 변환해 주는 함수 
    def tag_to_word(self, ner_predicts, answer): 
        for word, tag in ner_predicts:

            # 변환해야 하는 태그가 있는 경우 추가 
            if tag == 'B_FOOD': 
                answer = answer.replace(tag, word) 

        answer = answer.replace('}', '') 
        answer = answer.replace('}', '') 
        return answer 

In [16]:
# FindAnswer 클래스 테스트 코딩 
# 챗봇 엔진 전체 동작 과정 보여줌 - 챗봇 엔진 동작 코드 
from config.DBconfig import *
from utils.Database import Database
from utils.preprocess1 import Preprocess

# 전처리 객체 생성
p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dic.bin',
               userdic='./utils/user_dic.tsv')

# 질문/답변 학습 디비 연결 객체 생성
db = Database(
    host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME
)
db.connect()    # 디비 연결

# 원문
# query = "오전에 탕수육 10개 주문합니다"
# query = "화자의 질문 의도를 파악합니다."
# query = "안녕하세요"
query = "자장면 주문할게요"

# 의도 파악
from models.intent.IntentModel import IntentModel
intent = IntentModel(model_name='./models/intent/intent_model.h5', preprocess=p)
predict = intent.predict_class(query)
intent_name = intent.labels[predict]

# 개체명 인식
from models.ner.NerModel import NerModel
ner = NerModel(model_name='./models/ner/ner_model.h5', preprocess=p)
predicts = ner.predict(query)
ner_tags = ner.predict_tags(query)

print("질문 : ", query)
print("=" * 100)
print("의도 파악 : ", intent_name)
print("개체명 인식 : ", predicts)
print("답변 검색에 필요한 NER 태그 : ", ner_tags)
print("=" * 100)

# 답변 검색
# 답변 존재하지 않는 경우 예외 발생. 해당 문장 이해할 수 없다는 문장 출력 추가 
# 예외 사항이 발생하는 질문 내용들을 모델 학습 데이터로 활용
from utils.FindAnswer import FindAnswer

try:
    f = FindAnswer(db)
    answer_text, answer_image = f.search(intent_name, ner_tags)
    answer = f.tag_to_word(predicts, answer_text)
except:
    answer = "죄송해요 무슨 말인지 모르겠어요"

print("답변 : ", answer)

db.close() # 디비 연결 끊음

ERROR:root:(1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'from chatbot_train_data where intent = "주문" and ( ner like \'%B_FOOD%\' ) othe\' at line 1')
ERROR:root:(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'from chatbot_train_data where intent = '주문'  other by rand() limit 1' at line 1")


질문 :  자장면 주문할게요
의도 파악 :  주문
개체명 인식 :  [('자장면', 'B_FOOD'), ('주문', 'O')]
답변 검색에 필요한 NER 태그 :  ['B_FOOD']
답변 :  죄송해요 무슨 말인지 모르겠어요
